<a href="https://colab.research.google.com/github/damolathegreat/DamolaAgboola_Day8/blob/master/IBB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from scipy.optimize import minimize
import pandas as pd

class BiomethanolSystemOptimizer:
    def __init__(self):
        # System components data from your analysis
        self.components = {
            'PYRXTR': {
                'cost': 386508,
                'exergy_destroyed': 33.37,
                'unit_exergy_cost': 0.0150,
                'base_T': 200,
                'base_P': 1,
                'base_FR': 5440.16
            },
            'N2COMP': {
                'cost': 223928,
                'exergy_destroyed': 5.46,
                'unit_exergy_cost': 0.0200,
                'base_T': 200,
                'base_P': 1,
                'base_FR': 5448.90
            },
            'SOLID-GAS SEP': {
                'cost': 94086,
                'exergy_destroyed': 7.00,
                'unit_exergy_cost': 0.0120,
                'base_T': 200,
                'base_P': 1,
                'base_FR': 627.25
            },
            'SEPARATOR-1': {
                'cost': 105561,
                'exergy_destroyed': 100.10,
                'unit_exergy_cost': 0.0140,
                'base_T': 200,
                'base_P': 1,
                'base_FR': 6076.14
            },
            'COOLER': {
                'cost': 53031,
                'exergy_destroyed': 1205.89,
                'unit_exergy_cost': 0.0100,
                'base_T': 30,
                'base_P': 1,
                'base_FR': 6076.14
            },
            'COPRDTR-1': {
                'cost': 325233,
                'exergy_destroyed': 1092.71,
                'unit_exergy_cost': 0.0180,
                'base_T': 700,
                'base_P': 1,
                'base_FR': 10000.00
            }
            # Add other components as needed
        }

        # Economic parameters
        self.CRF = 0.106  # Capital Recovery Factor
        self.phi = 1.06   # Maintenance factor
        self.N = 8000     # Operating hours

    def calculate_total_cost(self, x):
        """
        Calculate total cost rate (Z + CD) for all components
        x: array of optimization variables [T1, P1, FR1, T2, P2, FR2, ...]
        """
        total_cost = 0
        n_vars_per_component = 3  # T, P, FR for each component

        for i, (name, data) in enumerate(self.components.items()):
            idx = i * n_vars_per_component
            T, P, FR = x[idx:idx + n_vars_per_component]

            # Calculate Z (capital investment rate)
            equipment_cost = data['cost'] * (T/data['base_T'])**0.2 * (P/data['base_P'])**0.3
            Z = (self.CRF * equipment_cost * self.phi)/(self.N * 3600)

            # Calculate CD (exergy destruction cost)
            exergy_destroyed = data['exergy_destroyed'] * (T/data['base_T'])**1.1 * (FR/data['base_FR'])**0.8
            CD = data['unit_exergy_cost'] * exergy_destroyed

            total_cost += Z + CD

        return total_cost

    def constraints(self, x):
        """Define system constraints"""
        constraints = []
        n_vars_per_component = 3

        for i, (name, data) in enumerate(self.components.items()):
            idx = i * n_vars_per_component
            T, P, FR = x[idx:idx + n_vars_per_component]

            # Temperature constraints
            if name == 'COOLER':
                constraints.extend([
                    100 - T,  # T ≤ 100°C for cooler
                    T - 20    # T ≥ 20°C for cooler
                ])
            elif name in ['COPRDTR-1', 'COPRDTR-2']:
                constraints.extend([
                    800 - T,  # T ≤ 800°C for reactors
                    T - 600   # T ≥ 600°C for reactors
                ])
            else:
                constraints.extend([
                    500 - T,  # T ≤ 500°C for other units
                    T - 100   # T ≥ 100°C for other units
                ])

            # Pressure constraints
            constraints.extend([
                5 - P,    # P ≤ 5 atm
                P - 1     # P ≥ 1 atm
            ])

            # Flow rate constraints
            constraints.extend([
                15000 - FR,  # FR ≤ 15000 kg/hr
                FR - 1000    # FR ≥ 1000 kg/hr
            ])

        return np.array(constraints)

    def optimize(self):
        """Perform optimization"""
        # Initial conditions
        x0 = []
        for _, data in self.components.items():
            x0.extend([data['base_T'], data['base_P'], data['base_FR']])

        # Optimization
        result = minimize(
            self.calculate_total_cost,
            x0,
            method='SLSQP',
            constraints={'type': 'ineq', 'fun': self.constraints},
            options={'maxiter': 1000}
        )

        return self.format_results(result)

    def format_results(self, result):
        """Format optimization results into DataFrame"""
        optimized_conditions = []
        n_vars_per_component = 3

        for i, (name, data) in enumerate(self.components.items()):
            idx = i * n_vars_per_component
            T, P, FR = result.x[idx:idx + n_vars_per_component]

            # Calculate costs with optimized conditions
            equipment_cost = data['cost'] * (T/data['base_T'])**0.2 * (P/data['base_P'])**0.3
            Z = (self.CRF * equipment_cost * self.phi)/(self.N * 3600)
            exergy_destroyed = data['exergy_destroyed'] * (T/data['base_T'])**1.1 * (FR/data['base_FR'])**0.8
            CD = data['unit_exergy_cost'] * exergy_destroyed

            optimized_conditions.append({
                'Component': name,
                'Temperature (°C)': round(T, 2),
                'Pressure (atm)': round(P, 2),
                'Flow Rate (kg/hr)': round(FR, 2),
                'Z ($/s)': round(Z, 4),
                'CD ($/s)': round(CD, 4),
                'Total Cost Rate ($/s)': round(Z + CD, 4)
            })

        return pd.DataFrame(optimized_conditions)

# Run optimization
optimizer = BiomethanolSystemOptimizer()
results = optimizer.optimize()
print("\nOptimization Results:")
print(results.to_string(index=False))


Optimization Results:
    Component  Temperature (°C)  Pressure (atm)  Flow Rate (kg/hr)  Z ($/s)  CD ($/s)  Total Cost Rate ($/s)
       PYRXTR             100.0             1.0            3468.38   0.0013    0.1629                 0.1642
       N2COMP             100.0             1.0            4979.99   0.0008    0.0474                 0.0482
SOLID-GAS SEP             100.0             1.0            1305.21   0.0003    0.0704                 0.0707
  SEPARATOR-1             100.0             1.0            1000.00   0.0004    0.1544                 0.1547
       COOLER              20.0             1.0            1000.00   0.0002    1.8227                 1.8228
    COPRDTR-1             600.0             1.0            1000.00   0.0012    2.6311                 2.6323
